In [148]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.metrics import mean_absolute_error as mae

In [17]:
# load data
train_data = pd.read_csv('/content/train_v2.csv',low_memory=False)


In [47]:
train_data

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,...,f740,f741,f742,f743,f744,f745,f746,f747,f748,f749,f750,f751,f752,f753,f754,f755,f756,f757,f758,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768,f769,f770,f771,f772,f773,f774,f775,f776,f777,f778,loss
0,1,126,10,0.686842,1100,3,13699,7201.0,4949.0,126.75,126.03,7,0.7607,0.7542,612922,0.7236,0.7236,0.5171,0.7236,0.8476,0.7876,1.097851e+09,89,66,998046.0,89.0,89.0,89.00,89.00,89.0,0.9271,0,0,0,5,0,0,0.73180,0.72124,0.749280,...,-4.40,6,9.76,-21.82,49.18,1.0435,-0.9066,1.5883,3.4154,2.8283,2.3451,0.030594,1.7418,1.5271,0.8474,0.4715,0.028362,3.1611,2.5162,2.0037,0.019636,4.4352,4.2676,-0.1524,1,-0.40,-0.560,-0.440,-0.6280,-3.14,5,2.14,-1.54,1.18,0.1833,0.7873,1,0,5,0
1,2,121,10,0.782776,1100,3,84645,240.0,1625.0,123.52,121.35,7,0.6555,0.6555,245815,0.6341,0.6341,0.3909,0.6667,0.6903,0.6903,8.449459e+08,78,50,754416.0,78.0,78.0,78.00,78.00,78.0,0.8478,0,0,0,6,0,0,0.65705,0.64069,0.626200,...,0.35,7,51.73,-72.44,798.74,2.7180,-0.8294,1.0740,2.9194,2.1180,1.5666,0.120442,1.1963,1.0322,0.4843,0.2389,0.130160,2.7659,1.9523,1.4059,0.115277,3.2763,2.7962,-0.3097,1,-0.17,-0.275,-0.203,-0.2300,-1.38,6,0.54,-0.24,0.13,0.1926,-0.6787,1,0,5,0
2,3,126,10,0.500080,1100,3,83607,1800.0,1527.0,127.76,126.49,7,0.7542,0.7542,1385872,0.7542,0.7542,0.5508,0.7542,0.8091,0.7807,1.308478e+09,89,54,1037651.0,89.0,89.0,100.43,94.37,89.0,0.9175,0,0,0,13,0,0,0.74710,0.75878,0.654685,...,-6.35,14,56.62,-79.48,621.97,1.9592,-0.3632,3.2969,7.2072,5.7062,4.5627,0.226336,3.3277,3.4166,1.8321,0.9979,0.103307,6.8623,5.2963,4.1282,0.219729,8.1381,7.3269,-0.1909,1,-0.58,-0.540,-0.572,-0.3985,-5.18,13,2.89,-1.73,1.04,0.2521,0.7258,1,0,5,0
3,4,134,10,0.439874,1100,3,82642,7542.0,1730.0,132.94,133.58,7,0.8017,0.7881,704687,0.7881,0.7881,0.5923,0.7881,0.8230,0.8158,1.472752e+09,93,55,1115721.0,93.0,93.0,114.63,102.92,93.0,0.9688,0,0,0,4,0,0,0.77405,0.78929,0.750675,...,3.75,5,36.31,18.15,429.81,2.5883,-0.8812,1.2950,2.5777,2.0830,1.6899,0.054630,1.3748,1.3421,0.7982,0.4810,0.081205,2.5571,2.0593,1.6653,0.056470,3.2516,3.0631,-0.1770,1,-0.75,-0.635,-0.745,-0.5100,-2.04,4,1.29,-0.89,0.66,0.2498,0.7119,1,0,5,0
4,5,109,9,0.502749,2900,4,79124,89.0,491.0,122.72,112.77,6,0.5263,0.5263,51985,0.5263,0.5263,0.3044,0.5405,0.5556,0.5455,1.442916e+09,60,21,536400.0,60.0,60.0,60.00,60.00,60.0,0.8451,0,0,0,26,0,0,0.60315,0.53367,0.696150,...,11.45,27,2206.14,-10559.05,708436.49,9.0293,-0.8760,6.3852,17.2599,14.2795,11.9179,0.085330,7.2175,6.2262,3.1446,1.6149,0.074286,15.9080,12.5688,9.9844,0.067540,17.5561,15.6079,-0.4444,1,-0.18,-0.280,-0.182,-0.4277,-11.12,26,6.11,-3.82,2.51,0.2282,-0.5399,0,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105466,105467,126,8,0.006620,2200,4,76831,331.0,2366.0,125.25,125.80,7,0.4878,0.4878,9281480,0.4918,0.4615,0.2147,0.4615,0.4800,0.4800,1.178496e+09,60,0,446400.0,60.0,60.0,73.96,66.40,60.0,0.5607,0,0,0,3,0,0,0.62075,0.49535,0.677967,...,-0.67,4,2.45,-0.30,2.20,0.7645,0.7937,0.8106,1.6156,1.2662,1.0186,0.167967,0.8336,0.8120,0.4839,0.2955,0.190560,1.5766,1.2164,0.9619,0.161290,1.9770,1.8424,-0.5200,1,0.00,-0.350,-0.050,-0.4600,-1.38,3,1.07,-0.92,0.82,0.3809,-0.5489,0,0,93,0
105467,105468,137,9,0.358568,2200,4,9836,3898.0,1106.0,135.88,136.99,10,0.7227,0.7049,901745,0.6880,0.6880,0.5147,0.6935,0.7818,0.7478,2.206863e+09,86,50,1013252.0,86.0,86.0,113.64,98.44,86.0,0.8776,0,0,0,3,0,0,0.64400,0.68920,0.623833,...,9.75,4,34.31,129.52,516.69,1.6236,-0.6855,0.5262,1.5442,1.1484,0.8691,0.117873,0.5842,0.6314,0.3230,0.1701,0.142930,1.4767,1.0764,0.7993,0.118383,1.7019,1.4648,-0.2182,1,-0.14,-0.210,-0.176,-0.2333,-0.70,3,0.33

In [46]:
#dealing categorical data
cols = list(train_data.columns)
categorical_index = []
for i in range(len(train_data.dtypes)):
  if train_data.dtypes[i] == object:
    categorical_index.append(i)
    

print(categorical_index)

[135, 136, 204, 205, 274, 275, 336, 388, 389, 417, 418, 465, 468, 528, 531, 617, 618, 686, 689]


In [172]:
cat_cols = []
for i in categorical_index:
  cat_cols.append(cols[i])


In [75]:
train_data = train_data.dropna()
X_num = train_data.drop(['id','loss'],axis =1)
X_num = X_num.drop(cat_cols, axis = 1)
X_cat = train_data[cat_cols]


In [161]:
np.shape(X_num)

(51940, 750)

In [149]:
X_c = []
for cat in cat_cols:
  enc = LabelEncoder()
  enc.fit(X_cat[cat].values.reshape(-1, 1))
  X_c.append(enc.transform(X_cat[cat].values.reshape(-1, 1)))




/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [156]:
x_cat = X_c[0]
for i in range(1,len(X_c)):
  x_cat=np.vstack((x_cat,X_c[i]))

In [160]:
X_cat = x_cat.transpose()
np.shape(X_cat)

(51940, 19)

In [163]:
X = np.hstack((X_num,X_cat))
np.shape(X)

(51940, 769)

In [164]:
X = X_num
y = train_data['loss']

X_train, X_cv, y_train, y_cv = train_test_split(X,y,test_size=0.3,random_state=47)

In [165]:
dtrain = xgb.DMatrix(X_train, label=y_train)
del(X_train)
dtest = xgb.DMatrix(X_cv)
del(X_cv)

In [166]:
#refer: https://www.kaggle.com/btyuhas/bayesian-optimization-with-xgboost
def xgb_evaluate(max_depth, gamma, colsample_bytree):
    params = {'eval_metric': 'rmse',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree}
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=3)    
    
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

In [167]:
xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7), 
                                             'gamma': (0, 1),
                                             'colsample_bytree': (0.3, 0.9)})
xgb_bo.maximize(init_points=3, n_iter=5, acq='ei')
params = xgb_bo.max['params']
params['max_depth'] = int(params['max_depth'])


|   iter    |  target   | colsam... |   gamma   | max_depth |
-------------------------------------------------------------
|  1        | -4.038    |  0.4389   |  0.4063   |  3.794    |
|  2        | -4.066    |  0.3565   |  0.2919   |  5.928    |
|  3        | -4.093    |  0.8002   |  0.7368   |  6.888    |
|  4        | -4.023    |  0.3      |  0.0      |  3.0      |
|  5        | -4.025    |  0.326    |  0.892    |  3.001    |
|  6        | -4.026    |  0.3221   |  0.03181  |  3.049    |
|  7        | -4.03     |  0.314    |  0.09335  |  3.014    |
|  8        | -4.044    |  0.3      |  1.0      |  4.752    |


In [168]:
d= xgb.DMatrix(X, label=y)
model2 = xgb.train(params, d, num_boost_round=250)
y_pred = model2.predict(dtest)
y_train_pred = model2.predict(d)



In [171]:
print('MAE ON Cross Validation DATA = ',mae(y_cv, y_pred))
print('MAE ON TRAIN DATA = ',mae(y, y_train_pred))

MAE ON Cross Validation DATA =  1.3177669758882988
MAE ON TRAIN DATA =  1.3148579868478645
